Cartar de aclaracion de posible desvío

In [1]:
import pandas as pd
import collections

from docxtpl import DocxTemplate

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [3]:
ruta

'C:\\Documents\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
path_xlsx_avance = r'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V3.xlsx'
#path_xlsx_avance = r'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/AVANCE DE COSECHA V3.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

C:\Users\bismarksr\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\bismarksr\anaconda3\envs\utea_reportes\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [6]:
xlsx_entregas.head(3)

,INST,GRUPO,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,PAQ,TM. ENTREGADAS,AVANCE,INICIO DE ENTREGAS,ULTIMA ENTREGA,COD_COSECHA
0,1,1,2710,DEMIQUEL JIMENEZ JOSELITO,1054.46,NaN,1482.15,1.405601,NaN,NaN,287.0
1,1,1,2751,DEMIQUEL TERESA JIMENEZ DE,114.10,NaN,120.89,1.059509,NaN,NaN,287.0
2,1,1,7572,LAVERAN VACA ZENON,64.75,NaN,42.35,0.654054,NaN,NaN,286.0


In [7]:
xlsx_avance.head(3)

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA
0,1,515,AGUILERA OLGA RIVERO VDA DE,26449.77,32139.67,1.215121,543.38,543.38,NaN,1.000000,...,0.215121,543.693782,55.390276,30115.348382,30115.348382,2024.321618,0.937015,NaN,-5689.90,5689.900000
1,2,1180,BALCAZAR BALCAZAR JESUS E.,10848.49,12360.22,1.139349,170.47,170.47,NaN,1.000000,...,0.139349,170.249063,50.971072,8677.777218,8677.777218,3682.442782,0.702073,NaN,-1511.73,1511.730000
2,3,911,BALCAZAR JUSTINIANO MARIO,5390.78,6233.97,1.156413,100.01,95.02,4.99,0.950105,...,0.206308,114.920802,47.592543,5469.373160,4540.358938,1693.611062,0.728325,248.458376,-843.19,1091.648376


## Argumentos

In [14]:
fecha_inf = '18 de noviembre de 2023'
fecha_form = '18-11-2023' # fecha para el nombre de archivo

fecha_aclaracion = '18 de noviembre de 2023'
cod_grupos = [474]

In [15]:
for i in cod_grupos:
    # datos del grupo
    grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == i].copy()
    grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
    grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
    grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
    
    # convertir datos del grupo a objetos
    list_obj_caneros = []
    for index, row in grupo.iterrows():
        canero = {
            'inst' : row['INST'],
            'cod_ca' : row['CODIGO CAÑERO '],
            'nom_ca' : row['NOMBRE CAÑERO'],
            'compromiso' : row['COMPROMISO DE ENTREGA'],
            'entrega' : row['TM. ENTREGADAS'],
            'entrega_porcen' : row['avance%']
        }
        list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
    
    # totales de los datos del grupo
    grupo_totales = {
        'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
        'entregas' : grupo['TM. ENTREGADAS'].sum(),
        'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
    }
    obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
    
    # datos generales de avance de cosecha
    avance = xlsx_avance[xlsx_avance['COD_COS'] == i]
    avance_totales = {
        'fecha_aclaracion' : fecha_aclaracion,
        'fecha_informe' : fecha_inf,
    
        'entrega_porcen' : float(avance['%_ENTREGA']) * 100,
        'ha_canha' : float(avance['TOTAL_AREA']),
        'ha_cosecha' : float(avance['COSECHA']),
        'cosecha_porcen' : float(avance['%COSECHA']) * 100,
        'diferencia_porcen' : float(avance['DIF.AVANCE']) * 100,
        
        'compromiso_rest' : float(avance['COMPROM_REST']),
        'tn_en_campo' : float(avance['TN_EN_CAMPO']),
        'diferencia' : float(avance['DIFERENCIA'])
    }
    
    # datos generales a objeto
    obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
    
    # generar reporte
    doc = DocxTemplate(ruta + '/templates/tpl_informe_aclarativa_desvio.docx')
    context = {
        'insp' : list_obj_caneros,
        'grupo_totales' : obj_grupo_totales,
        'avance' : obj_avance_totales
    }
    doc.render(context)
    doc.save(ruta + '/informes/_' + avance['NOMBRE CAÑERO'].iloc[0] + fecha_form + '.docx')
    print('Se generó reporte de grupo:', i)

Se generó reporte de grupo: 474


C:\Users\bismarksr\AppData\Local\Temp\ipykernel_60148\2495378206.py:35: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'entrega_porcen' : float(avance['%_ENTREGA']) * 100,
C:\Users\bismarksr\AppData\Local\Temp\ipykernel_60148\2495378206.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'ha_canha' : float(avance['TOTAL_AREA']),
C:\Users\bismarksr\AppData\Local\Temp\ipykernel_60148\2495378206.py:37: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  'ha_cosecha' : float(avance['COSECHA']),
C:\Users\bismarksr\AppData\Local\Temp\ipykernel_60148\2495378206.py:38: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instea

## DATOS DEL GRUPO

In [31]:
cod_grupo = 145

In [32]:
grupo = xlsx_entregas[xlsx_entregas['COD_COSECHA'] == cod_grupo].copy()
grupo = grupo[['INST', 'CODIGO CAÑERO ', 'NOMBRE CAÑERO','COMPROMISO DE ENTREGA', 'TM. ENTREGADAS']]
grupo['TM. ENTREGADAS'] = grupo['TM. ENTREGADAS'].fillna(0)
grupo['avance%'] = grupo['TM. ENTREGADAS'] / grupo['COMPROMISO DE ENTREGA'] * 100
grupo

,INST,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO DE ENTREGA,TM. ENTREGADAS,avance%
1339,304,2913,CALVIMONTES VARGAS ZAIRA EVELYN,895.06,241.02,26.927804
1340,304,14569,VARGAS SOTO ERWIN,781.63,1134.24,145.112137
1341,304,15146,VARGAS IGNACIA SOTO POLO DE,1899.38,1834.59,96.588887
1342,304,15308,VARGAS SOTO ELVIRA,591.64,884.67,149.528429


In [33]:
list_obj_caneros = []
for index, row in grupo.iterrows():
    canero = {
        'inst' : row['INST'],
        'cod_ca' : row['CODIGO CAÑERO '],
        'nom_ca' : row['NOMBRE CAÑERO'],
        'compromiso' : row['COMPROMISO DE ENTREGA'],
        'entrega' : row['TM. ENTREGADAS'],
        'entrega_porcen' : row['avance%']
    }
    list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
list_obj_caneros

[canero(inst=304, cod_ca=2913, nom_ca='CALVIMONTES VARGAS ZAIRA EVELYN ', compromiso=895.06, entrega=241.02, entrega_porcen=26.92780372265547),
 canero(inst=304, cod_ca=14569, nom_ca='VARGAS SOTO ERWIN ', compromiso=781.63, entrega=1134.24, entrega_porcen=145.11213745634123),
 canero(inst=304, cod_ca=15146, nom_ca='VARGAS IGNACIA SOTO POLO DE ', compromiso=1899.38, entrega=1834.59, entrega_porcen=96.58888689993576),
 canero(inst=304, cod_ca=15308, nom_ca='VARGAS SOTO ELVIRA', compromiso=591.64, entrega=884.67, entrega_porcen=149.5284294503414)]

In [34]:
grupo_totales = {
    'compromiso' : grupo['COMPROMISO DE ENTREGA'].sum(),
    'entregas' : grupo['TM. ENTREGADAS'].sum(),
    'entrega_porcen' : grupo['TM. ENTREGADAS'].sum() / grupo['COMPROMISO DE ENTREGA'].sum() * 100
}
obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
obj_grupo_totales

grupo_totales(compromiso=4167.71, entregas=4094.52, entrega_porcen=98.24387973251497)

## DATOS DE AVANCE

In [38]:
xlsx_avance.head(3)

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA
0,179,8016.0,PIL ANDINA S.A.,13206.87,8690.9,0.658059,358.47,270.66,87.81,0.755042,...,-0.096983,358.188662,51.34715,18391.966951,15789.085837,-7098.185837,1.816738,2602.881114,4515.97,-1913.088886
1,Grand Total,NaN,NaN,13206.87,8690.9,0.658059,358.47,270.66,87.81,0.755042,...,-0.096983,358.188662,51.34715,18391.966951,15789.085837,-7098.185837,1.816738,2602.881114,4515.97,-1913.088886
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
avance = xlsx_avance[xlsx_avance['COD_COS'] == cod_grupo]
avance

,COD_COS,CODIGO CAÑERO,NOMBRE CAÑERO,COMPROMISO,ENTREGA,%_ENTREGA,TOTAL_AREA,COSECHA,SIN COSECHA,%COSECHA,...,DIF.AVANCE,AREA_ESTIM.,TCH_ESTIM,PRODUCCION_ESTIM,TN_COSECHA_ESTIM,DIF.ESTIM,% DIF.ESTIM,TN_EN_CAMPO,COMPROM_REST,DIFERENCIA


In [45]:
avance.columns

Index(['COD_COS', 'CODIGO CAÑERO', 'NOMBRE CAÑERO', 'COMPROMISO', 'ENTREGA',
       '%_ENTREGA', 'TOTAL_AREA', 'COSECHA', 'SIN COSECHA', '%COSECHA', 'TCH',
       'DIF.AVANCE', 'AREA_ESTIM.', 'TCH_ESTIM', 'PRODUCCION_ESTIM',
       'TN_COSECHA_ESTIM', 'DIF.ESTIM', '% DIF.ESTIM', 'TN_EN_CAMPO',
       'COMPROM_REST', 'DIFERENCIA'],
      dtype='object')

In [46]:
avance_totales = {
    'fecha_aclaracion' : fecha_aclaracion,
    'fecha_informe' : fecha_inf,
    
    'entrega_porcen' : float(avance['%_ENTREGA']) * 100,
    'ha_canha' : float(avance['TOTAL_AREA']),
    'ha_cosecha' : float(avance['COSECHA']),
    'cosecha_porcen' : float(avance['%COSECHA']) * 100,
    'diferencia' : float(avance['DIF.AVANCE']) * 100,
    
    'compromiso_rest' : float(avance['COMPROM_REST']),
    'tn_en_campo' : float(avance['TN_EN_CAMPO']),
    'diferencia' : float(avance['DIFERENCIA'])
}
avance_totales

{'fecha_aclaracion': '06 de octubre de 2023',
 'fecha_informe': '07 de octubre de 2023',
 'entrega_porcen': 81.49958008487488,
 'ha_canha': 20.05,
 'ha_cosecha': 11.87,
 'cosecha_porcen': 59.20199501246882,
 'diferencia': -131.43524092747361,
 'compromiso_rest': 497.8499999999999,
 'tn_en_campo': 366.4147590725263}

In [74]:
avance['NOMBRE CAÑERO'].iloc[0]

'MARTINEZ RODRIGUEZ BASILIO'

In [21]:
obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
obj_avance_totales

avance_totales(fecha_aclaracion='10 de septiembre de 2023', fecha_informe='07 de octubre de 2023', entrega_porcen=98.72569780379943, ha_canha=543.39, ha_cosecha=420.85, cosecha_porcen=77.44897771398075, diferencia=5451.913216092134, compromiso_rest=337.0499999999993, tn_en_campo=5788.963216092133)

In [68]:
doc = DocxTemplate(ruta + '/templates/tbl_rpt_posible_desvio.docx')
context = {
    'insp' : list_obj_caneros,
    'grupo_totales' : obj_grupo_totales,
    'avance' : obj_avance_totales
}
doc.render(context)
doc.save(ruta + '/informes/rpt.docx')